- import the roboconnection library
- define the requester object
- setup handlers for messages

In [1]:
let rc = require('roboconnection')
let requester

- configure the connection info (api keys)

In [ ]:
let connectionInfo = {
    clientId: 'CLIENT_ID',
    clientSecret: 'SECRET_KEY',
    port: 7160
}

In [3]:
// EXAMPLE
let robotInfo = {
    serialName: 'Example-Example-Example-Example',
    endpoint: 'portal.jibo.com',
    email: 'EMAIL',
    password: 'PASSWORD',
}

- Instantiate the connection object

In [4]:
let jiboConnection = new rc.JiboConnection(robotInfo, connectionInfo, false);
console.log(`Ready to connect to: ${jiboConnection._robotInfo.serialName}`);

let onStatusMessage = (message) => {
    console.log(message.message);
}

let onDisconnected = () => {
    console.log(`onDisconnected`)
}

let onConnected = (req) => {
    requester = req;
    requester.play.say("hey, we're connected using a jupiter notebook!").complete;
}

jiboConnection.on('statusMessage', onStatusMessage);
jiboConnection.on('connected', onConnected);
jiboConnection.on('disconnected', onDisconnected);
console.log(`Listening for status messages from: ${jiboConnection._robotInfo.serialName}`);

Ready to connect to: Azure-Form-Caviar-Toile
Listening for status messages from: Azure-Form-Caviar-Toile


- connect to the robot
  - log messages will indicate the status of the connection process

In [5]:
jiboConnection.connect(false);
console.log(`Connecting to ${jiboConnection._robotInfo.serialName}...`);

Connecting to Azure-Form-Caviar-Toile...
login(): success
getRobotList(): success
createCertificate(): success
retrieveCertificate(): attempt #: 1
retrieveCertificate(): success: connecting to robot...
connectToRobot(): Connection: OK:


- test the api by sending a TTS (ESML) command

In [6]:
requester.expression.say("you look marvelous").complete;

- have Jibo turn to the left (relative to his starting position)

In [7]:
angleVector = {theta: -0.5, psi: 0};
lookAtTarget = { type: "ANGLE", angle: angleVector, levelHead: true};
requester.expression.look(lookAtTarget).complete;

{ Event: 'onLookAtAchieved',
  PositionTarget: [ 0, 0, 0 ],
  AngleTarget: [ 0, 0 ] }

- have Jibo look at an absolute position
  - CENTER {x: 1, y: 0, z: 0.5}
  - RIGHT {x: 0, y: -1, z: 0.4}
  - LEFT {x: 0, y: 1, z: 0.4}

In [8]:
vector = {x: 1, y: 0, z: 0.5}
position = { type: "POSITION", position: vector, levelHead: true};
requester.expression.look(position).complete;

{ Event: 'onLookAtAchieved',
  PositionTarget: [ 0, 0, 0 ],
  AngleTarget: [ 0, 0 ] }

- have Jibo ask a question

In [9]:
let p
let prompt

In [10]:
prompt = "How are you doing today?"
p = requester.expression.say(prompt).complete;
p.then( () => {
    let listenToken = requester.listen.start();
    listenToken.complete.then((status) => {
        console.log(`listenToken: complete:`, status);
    })
    .catch( (error) => {
        console.log(error);
    });
    listenToken.update.on((listenResultEvent) => {
            console.log("Hey! I think i heard something: ", listenResultEvent);
    });
})

p.catch( (error) => {
    console.log(error);
});

Hey! I think i heard something:  { Event: 'onListenResult',
  LanguageCode: 'en-US',
  Speech: 'hey thanks for asking I\'m doing great' }
listenToken: complete: { Event: 'onListenResult',
  LanguageCode: 'en-US',
  Speech: 'hey thanks for asking I\'m doing great' }


- have Jibo listen for Hey, Jibo

In [11]:
hjToken = requester.listen.subscribe.hotword();
hjToken.hotWordHeard.on((speaker) => {
    console.log("Hey! I heard Hey, Jibo: ", speaker);
});

e {
  name: 'Heard hot word',
  _onceEvents: Map {},
  _events: [ [Function] ],
  _maxListeners: 10,
  _emitterMode: 'ALL_HANDLERS' }

Hey! I heard Hey, Jibo:  { Event: 'onHotWordHeard',
  Speaker: 
   { LPSPosition: { Position: [Array], AngleVector: [Array], Confidence: 0 },
     SpeakerID: { Type: 'unknown', Confidence: 0 } } }


- disconnect

In [12]:
p = requester.play.say("goodbye for now").complete;
p.then( () => {
    jiboConnection.disconnect();
});


onDisconnected
disconnect()


#### API
```js
// Say
requester.expression.say("you look marvelous").complete;

// Listen
let listenToken = requester.listen.start();


// Look
let angleVector = {theta: 0.5, psi: 0};
let lookAtTarget = { type: "ANGLE", angle: angleVector, levelHead: true};
let p = requester.expression.look(lookAtTarget).complete;


// Look Vector New
let vector = {x: 1, y: 0, z: 0.5}
let position = { type: "POSITION", position: vector, levelHead: true};
let p = requester.expression.look(position).complete;

// Attention
let mode = "OFF"
let p = requester.expression.setAttention(mode as any).complete

//Config Volume
let configOptions: = { mixer: 1.0 };
let p = requester.config.set(configOptions).complete;

// Image Display New
let data:JIBO.v1.ImageView = {
    Type: "Image",
    Name: "imageName,
    Image: {
        name: "imageName,
        src: "url"
    }

}
let p = requester.display.swap(data).complete;


//Motion
motionTrackToken = requester.perception.subscribe.motion();
motionTrackToken.update.on((detectedMotions: any) => {});
motionTrackToken.cancel();

// Faces
let faceTrackToken = requester.perception.subscribe.face();
faceTrackToken.gained.on((detectedEntities: any) => {});
faceTrackToken.update.on((updatedEntities: any) => {});
faceTrackToken.cancel();


// Photo
let p = requester.media.capture.photo().complete;
p.then( (data: any) => {});

//Listen
let listenToken = requester.listen.start();
listenToken.update.on((listenResultEvent) => {});

// Hotword
hjToken = requester.listen.subscribe.hotword();
this.hjToken.hotWordHeard.on((speaker: any) => {});

```